In [2]:
import sys
print(sys.executable)

import openmm
print(openmm.__version__)

/home/anankaku/miniconda3/envs/openMM/bin/python
8.4


In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import urllib.request
import sys

# ==========================================================
# Step 1: Download Protein Structure (Villin Headpiece)
# ==========================================================
# This solves the FileNotFoundError by fetching the structure directly from RCSB PDB.
pdb_id = '1VII'
pdb_filename = 'villin.pdb'

print(f"Downloading protein structure ({pdb_id})...")
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb_id}.pdb', pdb_filename)
pdb = PDBFile(pdb_filename)

# Using Amber14 force field and TIP3P water model
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# ==========================================================
# Step 2: Set up the Simulation System
# ==========================================================
print("Creating the simulation system...")
# NonbondedMethod=NoCutoff is used for vacuum simulation (faster for testing)
system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff, constraints=HBonds)

# Integrator settings: 300K temperature, 2fs time step
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# ==========================================================
# Step 3: Energy Minimization
# ==========================================================
print("Performing energy minimization...")
simulation.minimizeEnergy()

# ==========================================================
# Step 4: Run 100 ps Production Simulation
# ==========================================================
# 100 ps / 0.002 ps = 50,000 steps
total_steps = 50000 
record_interval = 1000 # Save a frame every 1000 steps

# Reporters: Output trajectory (PDB) and simulation data (CSV-like format to console)
simulation.reporters.append(PDBReporter('output_100ps.pdb', record_interval))
simulation.reporters.append(StateDataReporter(sys.stdout, record_interval, step=True, 
                             potentialEnergy=True, temperature=True, speed=True, progress=True, totalSteps=total_steps))

print(f"Starting 100 ps simulation ({total_steps} steps)...")
simulation.step(total_steps)

print("-" * 30)
print("✅ Task Completed Successfully!")
print(f"Files generated: {pdb_filename} (initial), output_100ps.pdb (trajectory)")

Creating the simulation system...
Performing energy minimization...
Starting 100 ps simulation (50000 steps)...
#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
2.0%,1000,-2192.924808527566,282.9669581135361,0
4.0%,2000,-1979.5645244693412,296.94832961433946,28.1
6.0%,3000,-2058.1963472225757,292.8401815068089,28.5
8.0%,4000,-2042.5294487299075,276.4533332968049,28.3
10.0%,5000,-2094.325038216071,290.62419110091395,27.5
12.0%,6000,-2172.225884330115,303.08622166013106,27.6
14.0%,7000,-2241.105385026735,291.02993132433033,27.6
16.0%,8000,-2213.6408087400937,301.7120993653408,27.3
18.0%,9000,-2140.1033504348543,326.98210715030257,27.4
20.0%,10000,-2207.857374712801,278.70804482163686,27.5
22.0%,11000,-1976.067141890434,299.77508302648937,27.4
24.0%,12000,-2324.7957544468363,309.62372048562537,27.5
26.0%,13000,-2220.3900910987613,301.4928565830884,27.6
28.0%,14000,-2307.1524959355656,307.9162471149496,27.7
30.0%,15000,-2295.0906971132554,295.472070149